In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras import models, layers 
from sklearn.model_selection import train_test_split
import tensorflow as tf
import time
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical


In [2]:
# # Load the data
# X_train = np.load('histograms/reg_X_train.npy',mmap_mode='r')
# X_val = np.load('histograms/reg_X_val.npy',mmap_mode='r')
# # X_test = np.load('histograms/reg_X_test.npy',mmap_mode='r')
# y_train = np.load('histograms/reg_y_train.npy',mmap_mode='r')
# y_val = np.load('histograms/reg_y_val.npy',mmap_mode='r')
# # y_test = np.load('histograms/reg_y_test.npy',mmap_mode='r')

# ...existing code...
X_train = np.load('histograms/reg_X_train.npy', mmap_mode='r')
y_train = np.load('histograms/reg_y_train.npy', mmap_mode='r')
X_val   = np.load('histograms/reg_X_val.npy',   mmap_mode='r')
y_val   = np.load('histograms/reg_y_val.npy',   mmap_mode='r')

# def data_generator(x, y, batch_size):
#     num_samples = x.shape[0]
#     while True:
#         for i in range(0, num_samples, batch_size):
#             yield x[i:i+batch_size], y[i:i+batch_size]

# train_gen = data_generator(X_train, y_train, batch_size=32)
# val_gen   = data_generator(X_val, y_val, batch_size=32)


In [3]:
# def data_generator(x, y, batch_size, num_classes=4):
#     num_samples = x.shape[0]
#     while True:
#         for i in range(0, num_samples, batch_size):
#             x_batch = x[i:i+batch_size]
#             y_batch = y[i:i+batch_size]
#             y_batch_onehot = tf.keras.utils.to_categorical(y_batch, num_classes)
#             yield x_batch, y_batch_onehot

# train_gen = data_generator(X_train, y_train, batch_size=32)
# val_gen   = data_generator(X_val,   y_val,   batch_size=32)

def data_generator(features, labels, batch_size):
    num_samples = features.shape[0]
    while True:
        for i in range(0, num_samples, batch_size):
            x_batch = features[i:i+batch_size]
            y_batch = labels[i:i+batch_size]
            yield x_batch, y_batch

train_gen = data_generator(X_train, y_train, batch_size=32)
val_gen   = data_generator(X_val,   y_val,   batch_size=32)

In [4]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((420, 1500, 1500), (141, 1500, 1500), (420,), (141,))

In [5]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))

In [6]:
INPUT_SHAPE = (1500, 1500)

In [7]:
# Check if X_train and X_val are not None
if X_train is None or X_val is None:
    raise ValueError("X_train or X_val is None")

# Check if there are any NaN values in X_train and X_val
if np.isnan(X_train).any() or np.isnan(X_val).any():
    raise ValueError("X_train or X_val contains NaN values")

# Check the shapes of X_train and X_val
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

X_train shape: (420, 1500, 1500)
X_val shape: (141, 1500, 1500)


In [8]:
# Function to check dataset elements
def check_dataset(dataset, name):
    for i, (x, y) in enumerate(dataset):
        if x is None or y is None:
            raise ValueError(f"None value found in {name} at index {i}")
        
        # Check if x is a floating-point tensor before applying IsNan
        if x.dtype.is_floating:
            if tf.reduce_any(tf.math.is_nan(x)):
                raise ValueError(f"NaN value found in {name} at index {i} in features")
        
        # Check if y is a floating-point tensor before applying IsNan
        if y.dtype.is_floating:
            if tf.reduce_any(tf.math.is_nan(y)):
                raise ValueError(f"NaN value found in {name} at index {i} in labels")
        
        print(f"{name} element {i} - x shape: {x.shape}, y shape: {y.shape}")

# Assuming train_data and val_data are your TensorFlow datasets
check_dataset(train_data, "train_data")
check_dataset(val_data, "val_data")

train_data element 0 - x shape: (1500, 1500), y shape: ()
train_data element 1 - x shape: (1500, 1500), y shape: ()
train_data element 2 - x shape: (1500, 1500), y shape: ()
train_data element 3 - x shape: (1500, 1500), y shape: ()
train_data element 4 - x shape: (1500, 1500), y shape: ()
train_data element 5 - x shape: (1500, 1500), y shape: ()
train_data element 6 - x shape: (1500, 1500), y shape: ()
train_data element 7 - x shape: (1500, 1500), y shape: ()
train_data element 8 - x shape: (1500, 1500), y shape: ()
train_data element 9 - x shape: (1500, 1500), y shape: ()
train_data element 10 - x shape: (1500, 1500), y shape: ()
train_data element 11 - x shape: (1500, 1500), y shape: ()
train_data element 12 - x shape: (1500, 1500), y shape: ()
train_data element 13 - x shape: (1500, 1500), y shape: ()
train_data element 14 - x shape: (1500, 1500), y shape: ()
train_data element 15 - x shape: (1500, 1500), y shape: ()
train_data element 16 - x shape: (1500, 1500), y shape: ()
train_d

In [11]:
# model = models.Sequential()
# model.add(layers.Flatten())
# model.add(layers.Dense(4, activation='softmax'))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
# model.fit(
#     train_gen,
#     steps_per_epoch=len(X_train)//32,
#     validation_data=val_gen,
#     validation_steps=len(X_val)//32,
#     epochs=10
# )

Epoch 1/10


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 295ms/step - accuracy: 0.7043 - loss: 0.6597 - val_accuracy: 0.9453 - val_loss: 0.3577
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 262ms/step - accuracy: 1.0000 - loss: 0.1860 - val_accuracy: 0.9688 - val_loss: 0.3119
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 257ms/step - accuracy: 1.0000 - loss: 0.1789 - val_accuracy: 0.9688 - val_loss: 0.2938
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 254ms/step - accuracy: 1.0000 - loss: 0.1641 - val_accuracy: 0.9725 - val_loss: 0.2485
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 250ms/step - accuracy: 0.9990 - loss: 0.1329 - val_accuracy: 0.9633 - val_loss: 0.2972
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 259ms/step - accuracy: 0.9992 - loss: 0.1267 - val_accuracy: 0.9609 - val_loss: 0.2726
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 261ms/step - accuracy: 0.9994 - loss: 0.1159 - val_accuracy: 0.9922 - val_loss: 0.2274
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 252ms/step - accuracy: 0.9996 - loss: 0.1247 - val_accuracy: 1.0000 - val_

In [ ]:

# import time

# start_time = time.time()
# history = model.fit(
#     train_gen,
#     steps_per_epoch=len(X_train)//32,
#     validation_data=val_gen,
#     validation_steps=len(X_val)//32,
#     epochs=10
# )
# end_time = time.time()

# print("Training took:", end_time - start_time, "seconds")
# print("Train accuracy:", history.history["accuracy"])
# print("Validation accuracy:", history.history["val_accuracy"])

Epoch 1/10


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 250ms/step - accuracy: 0.9990 - loss: 0.0706 - val_accuracy: 0.9633 - val_loss: 0.2317
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 245ms/step - accuracy: 0.9992 - loss: 0.0710 - val_accuracy: 0.9609 - val_loss: 0.2147
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 248ms/step - accuracy: 0.9994 - loss: 0.0683 - val_accuracy: 0.9844 - val_loss: 0.1710
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 244ms/step - accuracy: 0.9996 - loss: 0.0793 - val_accuracy: 0.9922 - val_loss: 0.1645
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 234ms/step - accuracy: 1.0000 - loss: 0.0850 - val_accuracy: 0.9908 - val_loss: 0.1449
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 242ms/step - accuracy: 1.0000 - loss: 0.0853 - val_accuracy: 0.9633 - val_loss: 0.2011
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 251ms/step - accuracy: 1.0000 - loss: 0.0752 - val_accuracy: 0.9609 - val_loss: 0.1872
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 241ms/step - accuracy: 1.0000 - loss: 0.0657 - val_accuracy: 0.9844 - val_

In [9]:
# model.fit(X_train, epochs=10, validation_data= X_val)

In [9]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1500, 1500, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

start_time = time.time()
history = model.fit(
    train_gen,
    steps_per_epoch=len(X_train)//128,
    validation_data=val_gen,
    validation_steps=len(X_val)//128,
    epochs=10
)
end_time = time.time()

print("Training took:", end_time - start_time, "seconds")
print("Train accuracy:", history.history["accuracy"])
print("Validation accuracy:", history.history["val_accuracy"])
# ...existing code...

c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 4:25 133s/step - accuracy: 0.0938 - loss: 1.3870

: 

In [12]:
# Step 1: Define the CNN Model
model = models.Sequential([
    # Convolutional Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
    layers.MaxPooling2D((2, 2)),

    # Convolutional Layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Convolutional Layer 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Flatten the output
    layers.Flatten(),

    # Fully connected layer
    layers.Dense(128, activation='relu'),

    # Output layer (assuming 4 classes; adjust for your use case)
    layers.Dense(4, activation='softmax')  # Use 'sigmoid' for binary classification
])


# # ...existing code...
# X_train = np.expand_dims(X_train, axis=-1)
# X_val   = np.expand_dims(X_val,   axis=-1)

# y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=4)
# y_val_cat   = tf.keras.utils.to_categorical(y_val,   num_classes=4)

# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(1500, 1500, 1)),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Conv2D(64, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(4, activation='softmax')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(
#     X_train, y_train_cat,
#     epochs=10,
#     batch_size=32,
#     validation_data=(X_val, y_val_cat)
# )
# ...existing code...

c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


AbortedError: Graph execution error:

Detected at node StatefulPartitionedCall/gradient_tape/sequential_2_1/max_pooling2d_1_2/MaxPool2d/MaxPoolGrad defined at (most recent call last):
<stack traces unavailable>
Compute received an exception:Status:1, message: could not create a memory object. in file tensorflow/core/kernels/mkl/mkl_maxpooling_op.cc:388
	 [[{{node StatefulPartitionedCall/gradient_tape/sequential_2_1/max_pooling2d_1_2/MaxPool2d/MaxPoolGrad}}]] [Op:__inference_multi_step_on_iterator_10098]

In [ ]:
# Step 2: Compile the Model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'binary_crossentropy' for binary classification
              metrics=['accuracy'])


In [15]:
# Step 3: Train the Model
# Assuming `train_dataset` and `val_dataset` are TensorFlow datasets created from your data
model.fit(X_train, epochs=10, validation_data= X_val)

Epoch 1/10


ValueError: None values not supported.